In [1]:
import cv2
import numpy as np
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime

In [2]:
def pyramid(image, split_channels, initial_scale, final_scale, scale=1.5):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in split_channels],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
(winW, winH) = (120,24)
DS_min_winH_m=41
DS_max_winH_m=287
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m

clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

def show(img, x, y, winW, winH, positives=[]):
    clone = img.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def global_predict():
    global glob_X, glob_Y
    if len(glob_X)!=0:
        pred=[p[1] for p in clf.predict_proba(glob_X)]
        if len(glob_Y)==0:
            glob_Y=pred
        else:
            glob_Y=np.concatenate((glob_Y,pred))
        glob_X=[]
    
def predict(filename, scene_name, label_resolution, rects):
    global glob_X,glob_IDX,glob_IT,glob_DUMP_IT
    
    image=imread_resized(scene_name, filename, label_resolution)
    split_channels=read_split_channels(scene_name, filename)
    X=[]
    for resized_img, resized_split_ch, scale in pyramid(image, split_channels, initial_scale, final_scale, scale=1.15):
#         draw_preds=[]
        resized_ch=np.dstack(resized_split_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=4, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            features=channels_window.flatten()
            glob_X.append(features)
            glob_IDX.append([scene_name, filename, scale, (x,y)])
#             if clf.predict([features]):
#                 draw_preds.append((x, y))
            #show(resized_img, x, y, winW, winH, draw_preds)
    glob_IT+=1
    if glob_IT==glob_DUMP_IT:
        global_predict()
        glob_IT=0

In [4]:
glob_X=[]
glob_IDX=[]
glob_Y=[]
glob_IT=0
glob_DUMP_IT=1 # adjust manually to RAM

[predict(*row) for row in tqdm([row for row in walk_dataset() if row[1]=="blue_1"])]# and row[0]=="IMG_20200229_075416.jpg"])]
global_predict()

now = datetime.now()
date_time = now.strftime("%d-%m-%Y_%H:%M:%S")
np.save("run/sliding_window_IDX_"+date_time,glob_IDX)
np.save("run/sliding_window_Y_"+date_time,glob_Y)
print(len(glob_IDX), len(glob_Y),len(glob_X))

#glob_IDX=np.load("run/sliding_window_IDX_07-04-2020_17:59:14.npy",allow_pickle=True)
#glob_Y=np.load("run/sliding_window_Y_07-04-2020_17:59:14.npy",allow_pickle=True)
glob_Y=np.array([[e] for e in glob_Y])
RES=np.append(glob_IDX, glob_Y, 1)
print(np.array(glob_IDX).shape, glob_Y.shape, RES.shape)

100%|██████████| 48/48 [01:12<00:00,  1.52s/it]


2147040 2147040 0
(2147040, 4) (2147040, 1) (2147040, 5)


In [5]:
def area(rect):
    ((x,y),(x2,y2))=rect
    return (x2-x)*(y2-y)

def calc_overlapping(a,b):
    dx = min(a[1][0], b[1][0]) - max(a[0][0], b[0][0])
    dy = min(a[1][1], b[1][1]) - max(a[0][1], b[0][1])
    max_area=np.max([area(a), area(b)])
    overlap_area=dx*dy
    return overlap_area/max_area if ((dx>=0) and (dy>=0)) else 0

def calc_overlappings(rects, pred_rects):
    global glob_overlappings    
    overlappings=[np.max([calc_overlapping(pred_rect,rect) for pred_rect in pred_rects]) for rect in rects]
    glob_overlappings.extend(overlappings)

def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    img=imread_resized(scene_name, filename, label_resolution)
    file_res=RES[np.where((RES[:,0] == scene_name) * (RES[:,1] == filename))]
    
    n_rects=len(rects)
    max_rows=file_res[np.argsort(file_res[:,4])[-n_rects:]]
    pred_rects=[]
    rects=[add_margin(correct_rect_ratio(rect)) for rect in rects]
    for max_row in max_rows:
        [scene_name, filename, scale, (x,y), pred]=max_row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
        pred_rects.append(((x,y),(x+winW_s, y+winH_s)))
    for rect in rects:
        ((x,y),(x2,y2))=rect
        cv2.rectangle(img, (x,y), (x2,y2), (255, 0, 0), 2)
        
#     for row in file_res:
#         [scene_name, filename, scale, (x,y), pred]=row
#         [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
#         #print(winH_s)
#         if pred>0.9:
#             cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
    save_image(img, scene_name, filename, "predicted_labels")
    calc_overlappings(rects, pred_rects)

glob_overlappings=[]
[draw_predicted_rectangles(*row) for row in walk_dataset() if row[1]=="blue_1"]# and row[0]=="IMG_20200229_075416.jpg"]
print("Mean overlapping: ",np.mean(glob_overlappings))

IMG_20200229_075347.jpg
IMG_20200229_075348.jpg
IMG_20200229_075349.jpg
IMG_20200229_075350.jpg
IMG_20200229_075353.jpg
IMG_20200229_075356.jpg
IMG_20200229_075358.jpg
IMG_20200229_075400.jpg
IMG_20200229_075401.jpg
IMG_20200229_075405.jpg
IMG_20200229_075406.jpg
IMG_20200229_075407.jpg
IMG_20200229_075408.jpg
IMG_20200229_075410.jpg
IMG_20200229_075411.jpg
IMG_20200229_075413.jpg
IMG_20200229_075414.jpg
IMG_20200229_075416.jpg
IMG_20200229_075417.jpg
IMG_20200229_075419.jpg
IMG_20200229_075426.jpg
IMG_20200229_075427.jpg
IMG_20200229_075428.jpg
IMG_20200229_075430.jpg
IMG_20200229_075431.jpg
IMG_20200229_075434.jpg
IMG_20200229_075435.jpg
IMG_20200229_075436.jpg
IMG_20200229_075437.jpg
IMG_20200229_075438.jpg
IMG_20200229_075439.jpg
IMG_20200229_075440.jpg
IMG_20200229_075442.jpg
IMG_20200229_075443.jpg
IMG_20200229_075449.jpg
IMG_20200229_075450.jpg
IMG_20200229_075452.jpg
IMG_20200229_075453.jpg
IMG_20200229_075454.jpg
IMG_20200229_075457.jpg
IMG_20200229_075458.jpg
IMG_20200229_075

In [6]:
# window only - 1.36s
# features only - 1.47s
# classify per image - 2.82s
# classify per scale - 8.47s
# dsiplay, features only - 52.64s